In [97]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd
from sklearn import metrics

In [98]:
train_clicks = pd.read_csv('train_clicks.csv')
train_plans = pd.read_csv('train_plans.csv')
train_queries = pd.read_csv('train_queries.csv')
profiles = pd.read_csv('profiles.csv')

In [99]:
CP = pd.merge(train_clicks,train_plans, on='sid')

In [100]:
CPQ = pd.merge(CP,train_queries, on='sid')

In [101]:
# #get label
# label = CPQ['click_mode']

In [102]:
# #label을 합성한 자료에서 없애기
# CPQ = CPQ.drop(['click_mode'], axis=1)

In [103]:
#研究一下朋友寫的基本的前處理方式qq 什麽時候會好啊 我的技術。。
halfo = CPQ['o'].str.split(',',2,True)
halfd = CPQ['d'].str.split(',',2,True)
CPQ = CPQ.drop(['o','d'], axis=1)
CPQ = pd.concat([CPQ, halfo], axis = 1)
CPQ.rename(columns={ CPQ.columns[-1]: "o2" }, inplace=True)
CPQ.rename(columns={ CPQ.columns[-2]: "o1" }, inplace=True)
CPQ = pd.concat([CPQ, halfd], axis = 1)
CPQ.rename(columns={ CPQ.columns[-1]: "d2" }, inplace=True)
CPQ.rename(columns={ CPQ.columns[-2]: "d1" }, inplace=True)

In [104]:
CPQ.head(2)

,sid,click_time,click_mode,plan_time,plans,pid,req_time,o1,o2,d1,d2
0,2848914,2018-11-17 18:42:17,1,2018-11-17 12:56:15,"[{""distance"": 53156, ""price"": """", ""eta"": 6456,...",101804.0,2018-11-17 12:56:15,116.36,40.07,116.00,40.35
1,2629085,2018-10-12 16:28:13,3,2018-10-12 16:27:55,"[{""distance"": 13207, ""price"": 400, ""eta"": 2790...",203797.0,2018-10-12 16:27:55,116.35,40.08,116.33,40.03


In [105]:
#get od
CPQ_od = CPQ[['o1','o2','d1','d2']]
CPQ_od.head(2)

,o1,o2,d1,d2
0,116.36,40.07,116.00,40.35
1,116.35,40.08,116.33,40.03


In [106]:
o1 = CPQ_od['o1'].values.reshape(-1, 1)
o2 = CPQ_od['o2'].values.reshape(-1, 1)
d1 = CPQ_od['d1'].values.reshape(-1, 1)
d2 = CPQ_od['d2'].values.reshape(-1, 1)

In [107]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
scal_o1 = pd.DataFrame(scale.fit_transform(o1))
scal_o2 = pd.DataFrame(scale.fit_transform(o2))
scal_d1 = pd.DataFrame(scale.fit_transform(d1))
scal_d2 = pd.DataFrame(scale.fit_transform(d2))
scal_od = pd.concat([scal_o1, scal_o2], axis = 1)
scal_od = pd.concat([scal_od, scal_d1], axis = 1)
CPQ_od = pd.concat([scal_od, scal_d2], axis = 1)
CPQ_od.columns = ['o1', 'o2', 'd1', 'd2']
CPQ_od.head(2)

C:\Users\user\Anaconda3\envs\brent\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\user\Anaconda3\envs\brent\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\user\Anaconda3\envs\brent\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\user\Anaconda3\envs\brent\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\user\Anaconda3\envs\brent\lib\site-packages\sklearn\utils\validation.py:595: DataConver

,o1,o2,d1,d2
0,-0.240500,1.283963,-3.114013,3.733972
1,-0.327648,1.382996,-0.438127,0.790532


In [108]:
CPQ = CPQ[['sid', 'click_time', 'plan_time', 'plans', 'pid', 'req_time','click_mode']]

In [109]:
CPQ = pd.concat([CPQ, CPQ_od], axis = 1)
CPQ.head(2)

,sid,click_time,plan_time,plans,pid,req_time,click_mode,o1,o2,d1,d2
0,2848914,2018-11-17 18:42:17,2018-11-17 12:56:15,"[{""distance"": 53156, ""price"": """", ""eta"": 6456,...",101804.0,2018-11-17 12:56:15,1,-0.240500,1.283963,-3.114013,3.733972
1,2629085,2018-10-12 16:28:13,2018-10-12 16:27:55,"[{""distance"": 13207, ""price"": 400, ""eta"": 2790...",203797.0,2018-10-12 16:27:55,3,-0.327648,1.382996,-0.438127,0.790532


In [110]:
CPQ_Profiles = pd.merge(CPQ,profiles, on='pid')

In [111]:
CPQ_Profiles.head(2)

,sid,click_time,plan_time,plans,pid,req_time,click_mode,o1,o2,d1,...,p56,p57,p58,p59,p60,p61,p62,p63,p64,p65
0,2848914,2018-11-17 18:42:17,2018-11-17 12:56:15,"[{""distance"": 53156, ""price"": """", ""eta"": 6456,...",101804.0,2018-11-17 12:56:15,1,-0.240500,1.283963,-3.114013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2848913,2018-11-17 22:25:29,2018-11-17 22:25:16,"[{""distance"": 10634, ""price"": 2000, ""eta"": 235...",101804.0,2018-11-17 22:25:16,2,0.195237,-0.201522,0.048397,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [112]:
plans = CPQ_Profiles[['plans']]

In [113]:
plans.head(2)

,plans
0,"[{""distance"": 53156, ""price"": """", ""eta"": 6456,..."
1,"[{""distance"": 10634, ""price"": 2000, ""eta"": 235..."


In [114]:
plans = plans['plans'].str.replace('{"distance": ','')
plans = plans.str.replace('"price": ','')
plans = plans.str.replace('"eta": ','')
plans = plans.str.replace('"transport_mode": ','')
plans = plans.str.replace('}','')
plans = plans.str.replace('""','0')
plans = plans.str.replace('[','')
plans = plans.str.replace(']','')

In [115]:
plans.head(2)

0    53156, 0, 6456, 1, 48112, 0, 3535, 3, 48112, 1...
1    10634, 2000, 2355, 10, 10866, 0, 1917, 3, 1086...
Name: plans, dtype: object

In [116]:
#27跟沒有的差別在哪裏呢。。
plans = plans.str.split(',',27,True)
plans = plans.fillna(value=-1)

In [117]:
plans.columns = ['distance0', 'price0', 'eta0', 'transport_mode0','distance1', 'price1', 'eta1', 'transport_mode1','distance2', 'price2', 'eta2', 'transport_mode2','distance3', 'price3', 'eta3', 'transport_mode3','distance4', 'price4', 'eta4', 'transport_mode4','distance5', 'price5', 'eta5', 'transport_mode5','distance6', 'price6', 'eta6', 'transport_mode6']
plans.head(2)

,distance0,price0,eta0,transport_mode0,distance1,price1,eta1,transport_mode1,distance2,price2,...,eta4,transport_mode4,distance5,price5,eta5,transport_mode5,distance6,price6,eta6,transport_mode6
0,53156,0,6456,1,48112,0,3535,3,48112,16500,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,10634,2000,2355,10,10866,0,1917,3,10866,4700,...,2803,9,-1,-1,-1,-1,-1,-1,-1,-1


In [118]:
CPQ_Profiles = CPQ_Profiles.drop(['plans'], axis=1)
CPQ_Profiles = pd.concat([CPQ_Profiles, plans], axis = 1)
CPQ_Profiles.head()

,sid,click_time,plan_time,pid,req_time,click_mode,o1,o2,d1,d2,...,eta4,transport_mode4,distance5,price5,eta5,transport_mode5,distance6,price6,eta6,transport_mode6
0,2848914,2018-11-17 18:42:17,2018-11-17 12:56:15,101804.0,2018-11-17 12:56:15,1,-0.240500,1.283963,-3.114013,3.733972,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,2848913,2018-11-17 22:25:29,2018-11-17 22:25:16,101804.0,2018-11-17 22:25:16,2,0.195237,-0.201522,0.048397,0.422602,...,2803,9,-1,-1,-1,-1,-1,-1,-1,-1
2,2542469,2018-11-19 10:30:57,2018-11-19 10:30:50,101804.0,2018-11-19 10:30:50,2,1.938187,1.085898,0.210572,-0.221276,...,6228,1,-1,-1,-1,-1,-1,-1,-1,-1
3,381188,2018-10-05 12:37:07,2018-10-05 12:37:05,101804.0,2018-10-05 12:37:05,2,0.282385,-0.597652,1.589059,1.250444,...,6262,7,-1,-1,-1,-1,-1,-1,-1,-1
4,1601376,2018-10-03 13:32:30,2018-10-03 13:27:35,101804.0,2018-10-03 13:27:35,1,-0.153353,0.095575,0.048397,-0.497223,...,4013,1,-1,-1,-1,-1,-1,-1,-1,-1


In [119]:
###########################################################################################################################

In [120]:
CPQ_Profiles.columns

Index(['sid', 'click_time', 'plan_time', 'pid', 'req_time', 'click_mode', 'o1',
       'o2', 'd1', 'd2',
       ...
       'eta4', 'transport_mode4', 'distance5', 'price5', 'eta5',
       'transport_mode5', 'distance6', 'price6', 'eta6', 'transport_mode6'],
      dtype='object', length=104)

In [ ]:
#####################################ADD time differnce!!##########################################################

In [71]:
#get label
label = CPQ_Profiles['click_mode']
#label을 합성한 자료에서 없애기
CPQ_Profiles = CPQ_Profiles.drop(['sid', 'click_time', 'plan_time', 'pid', 'req_time', 'click_mode'], axis=1)

In [73]:
###########################################################################################################################

In [74]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
train_dataSV , test_dataSV , train_labelSV , test_labelSV = train_test_split(CPQ_Profiles,label,test_size=0.2)
tree = DecisionTreeClassifier(criterion = 'gini', random_state=5, max_depth=22)
tree.fit(train_dataSV, train_labelSV)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=22,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=5,
            splitter='best')

In [75]:
print(tree.predict(test_dataSV))
print(test_labelSV.values)
accuracyMlp = metrics.accuracy_score(train_labelSV.values, tree.predict(train_dataSV))
accuracy_testMlp = metrics.accuracy_score(test_labelSV.values, tree.predict(test_dataSV))
print(accuracyMlp)
print(accuracy_testMlp)

[9 7 2 ... 5 1 1]
[9 9 2 ... 5 1 3]
0.9160268227953242
0.7204142491329293


### Experiment

In [76]:
test_plans = pd.read_csv('test_plans.csv')
test_queries = pd.read_csv('test_queries.csv')
profiles = pd.read_csv('profiles.csv')

In [86]:
RCPQ = pd.merge(test_plans,test_queries, on='sid')
Rhalfo = RCPQ['o'].str.split(',',2,True)
Rhalfd = RCPQ['d'].str.split(',',2,True)
RCPQ = RCPQ.drop(['o','d'], axis=1)
RCPQ = pd.concat([RCPQ, Rhalfo], axis = 1)
RCPQ.rename(columns={ RCPQ.columns[-1]: "o2" }, inplace=True)
RCPQ.rename(columns={ RCPQ.columns[-2]: "o1" }, inplace=True)
RCPQ = pd.concat([RCPQ, Rhalfd], axis = 1)
RCPQ.rename(columns={ RCPQ.columns[-1]: "d2" }, inplace=True)
RCPQ.rename(columns={ RCPQ.columns[-2]: "d1" }, inplace=True)
RCPQ_od = RCPQ[['o1','o2','d1','d2']]
o1 = RCPQ_od['o1'].values.reshape(-1, 1)
o2 = RCPQ_od['o2'].values.reshape(-1, 1)
d1 = RCPQ_od['d1'].values.reshape(-1, 1)
d2 = RCPQ_od['d2'].values.reshape(-1, 1)
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
scal_o1 = pd.DataFrame(scale.fit_transform(o1))
scal_o2 = pd.DataFrame(scale.fit_transform(o2))
scal_d1 = pd.DataFrame(scale.fit_transform(d1))
scal_d2 = pd.DataFrame(scale.fit_transform(d2))
scal_od = pd.concat([scal_o1, scal_o2], axis = 1)
scal_od = pd.concat([scal_od, scal_d1], axis = 1)
RCPQ_od = pd.concat([scal_od, scal_d2], axis = 1)
RCPQ_od.columns = ['o1', 'o2', 'd1', 'd2']
##################
RCPQ = RCPQ[['sid', 'plan_time', 'plans', 'pid', 'req_time']]
RCPQ = pd.concat([RCPQ, RCPQ_od], axis = 1)
RCPQ_Profiles = pd.merge(RCPQ,profiles, on='pid')

Rplans = RCPQ_Profiles[['plans']]

Rplans = Rplans['plans'].str.replace('{"distance": ','')
Rplans = Rplans.str.replace('"price": ','')
Rplans = Rplans.str.replace('"eta": ','')
Rplans = Rplans.str.replace('"transport_mode": ','')
Rplans = Rplans.str.replace('}','')
Rplans = Rplans.str.replace('""','0')
Rplans = Rplans.str.replace('[','')
Rplans = Rplans.str.replace(']','')

Rplans = Rplans.str.split(',',27,True)
Rplans = Rplans.fillna(value=-1)

Rplans.columns = ['distance0', 'price0', 'eta0', 'transport_mode0','distance1', 'price1', 'eta1', 'transport_mode1','distance2', 'price2', 'eta2', 'transport_mode2','distance3', 'price3', 'eta3', 'transport_mode3','distance4', 'price4', 'eta4', 'transport_mode4','distance5', 'price5', 'eta5', 'transport_mode5','distance6', 'price6', 'eta6', 'transport_mode6']

RCPQ_Profiles = RCPQ_Profiles.drop(['plans'], axis=1)
RCPQ_Profiles = pd.concat([RCPQ_Profiles, Rplans], axis = 1)

sid = RCPQ_Profiles[['sid']]
RCPQ_Profiles = RCPQ_Profiles.drop(['sid', 'plan_time', 'pid', 'req_time'], axis=1)

C:\Users\user\Anaconda3\envs\brent\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\user\Anaconda3\envs\brent\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\user\Anaconda3\envs\brent\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\user\Anaconda3\envs\brent\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\user\Anaconda3\envs\brent\lib\site-packages\sklearn\utils\validation.py:595: DataConver

In [87]:
result = pd.DataFrame(tree.predict(RCPQ_Profiles))

In [88]:
result = result.astype('int')

In [89]:
result = pd.concat([sid, result], axis=1)

In [94]:
result.head(3)

,sid,0
0,1413458,2
1,1413461,2
2,2041251,7


In [92]:
result.to_csv('Result_0512.csv')

In [84]:
RCPQ_Profiles.head(1)

,o1,o2,d1,d2,p0,p1,p2,p3,p4,p5,...,eta4,transport_mode4,distance5,price5,eta5,transport_mode5,distance6,price6,eta6,transport_mode6
0,0.932843,-0.090398,1.074481,-1.431953,1.0,0.0,0.0,0.0,0.0,0.0,...,8397,1,-1,-1,-1,-1,-1,-1,-1,-1


In [85]:
CPQ_Profiles.head(1)

,o1,o2,d1,d2,p0,p1,p2,p3,p4,p5,...,eta4,transport_mode4,distance5,price5,eta5,transport_mode5,distance6,price6,eta6,transport_mode6
0,-0.2405,1.283963,-3.114013,3.733972,0.0,0.0,1.0,0.0,0.0,0.0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
